In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from my_functions import *
import seaborn as sns
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))

In [ ]:
## Load QSO catalog
filename = ('/home/alberto/cosmos/JPAS_mocks_sep2021/'
    'JPAS_mocks_classification_01sep_model11/Fluxes/Qso_jpas_mock_flam_train.cat')

my_filter_order = np.arange(60)
my_filter_order[[-4, -3, -2, -1]] = np.array([1, 12, 28, 43])
my_filter_order[1:-4] += 1
my_filter_order[12:-4] += 1
my_filter_order[28:-4] += 1
my_filter_order[43:-4] += 1

pm_flx = pd.read_csv(
    filename, sep=' ', usecols=range(2, 2 + 60)
).to_numpy().T[my_filter_order]
pm_err = pd.read_csv(
    filename, sep=' ', usecols=range(2 + 60, 2 + 60 + 60)
).to_numpy().T[my_filter_order]
zspec = pd.read_csv(filename, sep=' ', usecols=[127]).to_numpy().reshape(-1, )

# plate = pd.read_csv(filename, sep=' ', usecols=[122]).to_numpy().reshape(-1, )
# mjd = pd.read_csv(filename, sep=' ', usecols=[123]).to_numpy().reshape(-1, )
# fiber = pd.read_csv(filename, sep=' ', usecols=[124]).to_numpy().reshape(-1, )

w_central = central_wavelength()

In [ ]:
Lya_fts = pd.read_csv('csv/Lya_fts.csv')
EW_lya = Lya_fts.LyaEW

In [ ]:
w_lya = 1215.67 # A

N_sources = pm_flx.shape[1]
z_nb_Arr = w_central[:-4] / w_lya - 1
print(N_sources)

In [ ]:
mag = flux_to_mag(pm_flx, w_central.reshape(-1, 1))
mag[np.isnan(mag)] = 99.

In [ ]:
zspec_dist = cosmo.luminosity_distance(zspec).to(u.cm).value
L = EW_lya * Lya_fts.LyaCont * 1e-17 * 4*np.pi * zspec_dist**2
L = np.log10(L)
L[np.isnan(L)] = -99

In [ ]:
# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(pm_flx, pm_err)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(pm_flx, pm_err, IGM_T_correct=False)

In [ ]:
ew0_min = 0
ew0_max = 50
ew0_steps = 11
r_min = 18
r_max = 24
r_bins = 30
L_min = 43
L_max = 45.5
L_bins = 30
selection_r_Mat = np.zeros((ew0_steps, r_bins))
selection_L_Mat = np.zeros((ew0_steps, L_bins))
rightz_r_Mat = np.zeros((ew0_steps, r_bins))
rightz_L_Mat = np.zeros((ew0_steps, L_bins))
bins_r = np.linspace(r_min, r_max, r_bins + 1)
bins_L = np.linspace(L_min, L_max, L_bins + 1)

line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, 30)
lya_lines, lya_cont_lines = identify_lines(line, pm_flx, pm_err, first=True)
z_nb_Arr = np.ones(N_sources) * -1 
z_nb_Arr[np.where(np.array(lya_lines) != -1)] =\
    z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])

for i, ew0min in enumerate(np.linspace(ew0_min, ew0_max, ew0_steps)):
    print(ew0min)
    
    line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other, ew0min)
    other_lines = identify_lines(line_other, pm_flx, pm_err, first=False)

    nice_lya = nice_lya_select(lya_lines, other_lines, pm_flx, cont_est_other, z_nb_Arr)

    selection_r_Mat[i], _ = np.histogram(mag[-2, nice_lya], bins=bins_r)
    rightz_r_Mat[i], _ = np.histogram(
        mag[-2, nice_lya & (np.abs(z_nb_Arr - zspec) < 0.12)], bins=bins_r
    )

    selection_L_Mat[i], _ = np.histogram(L[nice_lya], bins=bins_L)
    rightz_L_Mat[i], _ = np.histogram(
        L[nice_lya & (np.abs(z_nb_Arr - zspec) < 0.12)], bins=bins_L
    )

target = (EW_lya / (1 + zspec) > 5) & (zspec > 2) 

hist_r, _ = np.histogram(mag[-2, target], bins=bins_r)
hist_L, _ = np.histogram(L[target], bins=bins_L)

In [ ]:
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.01


rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom + height + spacing, width, 0.2]
rect_histy = [left + width + spacing, bottom, 0.2, height]
rect_cbar = [left + width + spacing + 0.2 + 4*spacing, bottom, 0.05, height]

fig = plt.figure(figsize=(8, 8))

ax = fig.add_axes(rect_scatter)
ax_histx = fig.add_axes(rect_histx)
ax_histy = fig.add_axes(rect_histy)
ax_cbar = fig.add_axes(rect_cbar)

sns.heatmap(rightz_r_Mat / hist_r, vmin=0, vmax=1, ax=ax, cbar_ax=ax_cbar)
ax_histx.step(np.arange(r_bins), hist_r)
ax_histy.plot(np.sum(rightz_r_Mat, axis=1) / np.sum(hist_r), np.arange(ew0_steps))

ax_histx.tick_params(bottom=False, labelbottom=False)
ax_histy.tick_params(left=False, labelleft=False)

ax.invert_yaxis()

xticks = np.arange(r_bins, step=1.5)
xticks_labels = ['{0:0.1f}'.format((r_max - r_min) * x / r_bins + r_min) for x in xticks]
yticks_labels = ['{0:0.1f}'.format(x) for x in np.linspace(ew0_min, ew0_max, ew0_steps)]

ax.set_xticks(xticks)
ax.set_xticklabels(xticks_labels)
ax.set_yticklabels(yticks_labels)

ax_histy.set_title('Total completeness')
ax_histx.set_title('All sources')

ax.set_xlabel('rSDSS', fontsize=15)
ax.set_ylabel('Minimum EW$_0$ ($\AA$)', fontsize=15)

plt.show()

In [ ]:
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.01


rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom + height + spacing, width, 0.2]
rect_histy = [left + width + spacing, bottom, 0.2, height]
rect_cbar = [left + width + spacing + 0.2 + 4*spacing, bottom, 0.05, height]

fig = plt.figure(figsize=(8, 8))

ax = fig.add_axes(rect_scatter)
ax_histx = fig.add_axes(rect_histx)
ax_histy = fig.add_axes(rect_histy)
ax_cbar = fig.add_axes(rect_cbar)

sns.heatmap(rightz_r_Mat / selection_r_Mat, vmin=0, vmax=1, ax=ax, cbar_ax=ax_cbar)
ax_histx.step(np.arange(r_bins), hist_r)
ax_histy.plot(np.sum(rightz_r_Mat, axis=1) / np.sum(selection_r_Mat, axis=1),
    np.arange(ew0_steps))

ax_histx.tick_params(bottom=False, labelbottom=False)
ax_histy.tick_params(left=False, labelleft=False)

ax.invert_yaxis()

xticks = np.arange(r_bins, step=1.5)
xticks_labels = ['{0:0.1f}'.format((r_max - r_min) * x / r_bins + r_min) for x in xticks]
yticks_labels = ['{0:0.1f}'.format(x) for x in np.linspace(ew0_min, ew0_max, ew0_steps)]

ax.set_xticks(xticks)
ax.set_xticklabels(xticks_labels)
ax.set_yticklabels(yticks_labels)

ax_histy.set_title('Total purity')
ax_histx.set_title('All sources')

ax.set_xlabel('rSDSS', fontsize=15)
ax.set_ylabel('Minimum EW$_0$ ($\AA$)', fontsize=15)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.step(np.linspace(r_min, r_max, r_bins), selection_r_Mat[3], label='Selection')
ax.step(np.linspace(r_min, r_max, r_bins), hist_r, color='red', label='All')
ax.step(np.linspace(r_min, r_max, r_bins), rightz_r_Mat[3], label='Correct z')
ax.set_ylim(0, 2200)
ax.set_xlabel('rSDSS', fontsize=15)
ax.legend(fontsize=15, loc=2)
plt.show()

fig, ax = plt.subplots(figsize=(8,6))
ax.step(np.linspace(r_min, r_max, r_bins), np.cumsum(selection_r_Mat[3]), label='Selection')
ax.step(np.linspace(r_min, r_max, r_bins), np.cumsum(hist_r), color='red', label='All')
ax.step(np.linspace(r_min, r_max, r_bins), np.cumsum(rightz_r_Mat[3]), label='Correct z')
ax.set_xlabel('rSDSS', fontsize=15)
ax.legend(fontsize=15, loc=2)
plt.show()

In [ ]:
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.01


rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom + height + spacing, width, 0.2]
rect_histy = [left + width + spacing, bottom, 0.2, height]
rect_cbar = [left + width + spacing + 0.2 + 4*spacing, bottom, 0.05, height]

fig = plt.figure(figsize=(8, 8))

ax = fig.add_axes(rect_scatter)
ax_histx = fig.add_axes(rect_histx)
ax_histy = fig.add_axes(rect_histy)
ax_cbar = fig.add_axes(rect_cbar)

sns.heatmap(rightz_L_Mat / hist_L, vmin=0, vmax=1, ax=ax, cbar_ax=ax_cbar)
ax_histx.step(np.arange(L_bins), hist_L)
ax_histy.plot(np.sum(rightz_L_Mat, axis=1) / np.sum(hist_L), np.arange(ew0_steps))

ax_histx.tick_params(bottom=False, labelbottom=False)
ax_histy.tick_params(left=False, labelleft=False)

ax.invert_yaxis()

xticks = np.arange(r_bins, step=1.5)
xticks_labels = ['{0:0.1f}'.format((L_max - L_min) * x / L_bins + L_min) for x in xticks]
yticks_labels = ['{0:0.1f}'.format(x) for x in np.linspace(ew0_min, ew0_max, ew0_steps)]

ax.set_xticks(xticks)
ax.set_xticklabels(xticks_labels)
ax.set_yticklabels(yticks_labels)

ax_histy.set_title('Total completeness')
ax_histx.set_title('All sources')

ax.set_xlabel('log L', fontsize=15)
ax.set_ylabel('Minimum EW$_0$ ($\AA$)', fontsize=15)

plt.show()

In [ ]:
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.01


rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom + height + spacing, width, 0.2]
rect_histy = [left + width + spacing, bottom, 0.2, height]
rect_cbar = [left + width + spacing + 0.2 + 4*spacing, bottom, 0.05, height]

fig = plt.figure(figsize=(8, 8))

ax = fig.add_axes(rect_scatter)
ax_histx = fig.add_axes(rect_histx)
ax_histy = fig.add_axes(rect_histy)
ax_cbar = fig.add_axes(rect_cbar)

sns.heatmap(rightz_L_Mat / selection_L_Mat, vmin=0, vmax=1, ax=ax, cbar_ax=ax_cbar)
ax_histx.step(np.arange(L_bins), hist_L)
ax_histy.plot(np.sum(rightz_L_Mat, axis=1) / np.sum(selection_L_Mat, axis=1),
    np.arange(ew0_steps))

ax_histx.tick_params(bottom=False, labelbottom=False)
ax_histy.tick_params(left=False, labelleft=False)

ax.invert_yaxis()

xticks = np.arange(L_bins, step=1.5)
xticks_labels = ['{0:0.1f}'.format((L_max - L_min) * x / L_bins + L_min) for x in xticks]
yticks_labels = ['{0:0.1f}'.format(x) for x in np.linspace(ew0_min, ew0_max, ew0_steps)]

ax.set_xticks(xticks)
ax.set_xticklabels(xticks_labels)
ax.set_yticklabels(yticks_labels)

ax_histy.set_title('Total purity')
ax_histx.set_title('All sources')

ax.set_xlabel('log L', fontsize=15)
ax.set_ylabel('Minimum EW$_0$ ($\AA$)', fontsize=15)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.step(np.linspace(L_min, L_max, L_bins),selection_L_Mat[3], label='Selection')
ax.step(np.linspace(L_min, L_max, L_bins), hist_L, color='red', label='All')
ax.step(np.linspace(L_min, L_max, L_bins), rightz_L_Mat[3], label='Correct z')
#ax.hist(mag[-2], bins=bins_r, color='green', histtype='step')
ax.set_ylim(0)
ax.legend(fontsize=15, loc=2)
plt.show()

fig, ax = plt.subplots(figsize=(8,6))
ax.step(np.linspace(L_min, L_max, L_bins), np.cumsum(selection_L_Mat[3]), label='Selection')
ax.step(np.linspace(L_min, L_max, L_bins), np.cumsum(hist_L), color='red', label='All')
ax.step(np.linspace(L_min, L_max, L_bins), np.cumsum(rightz_L_Mat[3]), label='Correct z')
#ax.hist(mag[-2], bins=bins_r, color='green', histtype='step')
ax.set_ylim(0)
ax.legend(fontsize=15, loc=2)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

ax.hist2d(zspec[nice_lya], z_nb_Arr[nice_lya],
    bins=(np.linspace(1.5, 3.5, 40), np.linspace(2, 4, 40)))
plt.show()

In [ ]:
plt.hist(EW_lya[nice_lya & (np.abs(z_nb_Arr - zspec) < 0.12) & (zspec > 1.5)],
    bins=np.linspace(0, 500, 30))
plt.show()